In [1]:
//q6 start
import java.io._
def localpath(path: String): String = {
    "file://" + new java.io.File(".").getCanonicalPath + "/" + path
}
// `val sc` is already introduced in the namespace.
//val stack_posts_lines = sc.textFile(localpath("spark-stack-data/allPosts/"))
var valid_stack_posts_lines=sc.textFile(localpath("spark-stats-data/allPosts/"))
.filter(x => x.endsWith(" />") && x.startsWith("  <row"))
//val valid_stack_posts_lines = stack_posts_lines.filter(x => x.endsWith(" />") && x.startsWith("  <row"))
//val stack_users_lines = sc.textFile(localpath("spark-stack-data/allUsers/"))
var valid_stack_users_lines = sc.textFile(localpath("spark-stats-data/allUsers/"))
.filter(x => x.endsWith(" />") && x.startsWith("  <row"))
//val valid_stack_users_lines = stack_users_lines.filter(x => x.endsWith(" />") && x.startsWith("  <row"))
//valid_stack_users_lines.count()

Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-saman-2emonjezi-40gmail-2ecom:4042
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1550505297342)
SparkSession available as 'spark'


import java.io._
localpath: (path: String)String
valid_stack_posts_lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:33
valid_stack_users_lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at filter at <console>:37


In [2]:
//exact numbers in notebook
valid_stack_posts_lines.filter(x => x.contains("PostTypeId=\"2\"")).count()
valid_stack_posts_lines.filter(x => x.contains("PostTypeId=\"1\"")).count()

res0: Long = 52060


In [2]:
// NEW part 1
////q6 user creation date info new
import java.sql.Timestamp
import java.text.SimpleDateFormat

val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") //hh for 24 format, Z for time zone such as -0800
val day = 24*60*60*1000.0

def successMatches2(s: String): (Int,Float) = {
  val IdMatcher = """ Id=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val Id = IdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()/day
  (Id.toInt,time.toFloat)
  }
//q6 dates to df
val q6_creationdate=valid_stack_users_lines.map(x => successMatches2(x))
//.map(line => (line._1,line._2,line._3)).sortBy(_._2, ascending=false).collect not this
.map(line => (line._1,line._2))
var cols2 : Array[String] = Array("Id", "cdate");
val df_q6_cdate= q6_creationdate.toDF(cols2: _*);
//var df_q6_cdate= valid_stack_users_lines.map(x => successMatches2(x))
//.map(line => (line._1,line._2,line._3)).collect.toSeq.toDF(cols2: _*)

import java.sql.Timestamp
import java.text.SimpleDateFormat
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
day: Double = 8.64E7
successMatches2: (s: String)(Int, Float)
q6_creationdate: org.apache.spark.rdd.RDD[(Int, Float)] = MapPartitionsRDD[7] at map at <console>:50
cols2: Array[String] = Array(Id, cdate)
df_q6_cdate: org.apache.spark.sql.DataFrame = [Id: int, cdate: float]


In [3]:
//NEW part2
//q6 start, questions
import java.text.SimpleDateFormat
val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") 
val day = 24*60*60*1000.0
def successMatches1(s: String): (String,Float,Int,Int,Int,Int) = {
  val OwnerUserIdMatcher = """ OwnerUserId=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val AnswerCountMatcher = """ AnswerCount=\"(.*?)\" """.r
  val ViewCountMatcher = """ ViewCount=\"(.*?)\" """.r
  val ScoreMatcher = """ Score=\"(\d+)\" """.r
  val FavoriteCountMatcher = """ FavoriteCount=\"(\d+)\" """.r
  val AnswerCount = AnswerCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val FavoriteCount = FavoriteCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val ViewCount = ViewCountMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val OwnerUserId = OwnerUserIdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val Score = ScoreMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()/day 
  (OwnerUserId,time.toFloat,Score.toInt,FavoriteCount.toInt,ViewCount.toInt,AnswerCount.toInt)
  }
val valid_stack_posts_lines_questions = valid_stack_posts_lines
.filter(x => x.contains(" PostTypeId=\"1\""))// && x.contains("OwnerUserId=\""))

//all first questions
val q6_first_qs=valid_stack_posts_lines_questions.map(x => successMatches1(x))
.map(line => (line._1,(line._2,line._3,line._4,line._5,line._6))).reduceByKey {
  case ((dateL,favL,scL,vcL,acL), (dateR,favR,scR,vcR,acR)) => 
    (if (dateL >= dateR) (dateR,favR,scR,vcR,acR) else (dateL,favL,scL,vcL,acL))
}.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4,line._2._5))
var cols1 : Array[String] = Array("Id", "cdate","Score","FavoriteCount","ViewCount","AnswerCount")
val df_q6_user_info= q6_first_qs.toDF(cols1: _*)
//var df_q6_user_info= valid_stack_posts_lines_questions.map(x => successMatches1(x))
//.map(line => (line._1,(line._2,line._3,line._4,line._5,line._6))).reduceByKey {
//  case ((dateL,favL,scL,vcL,acL), (dateR,favR,scR,vcR,acR)) => 
//    (if (dateL >= dateR) (dateR,favR,scR,vcR,acR) else (dateL,favL,scL,vcL,acL))
//}.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4,line._2._5))
//.toDF(cols1: _*);

import java.text.SimpleDateFormat
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
day: Double = 8.64E7
successMatches1: (s: String)(String, Float, Int, Int, Int, Int)
valid_stack_posts_lines_questions: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[8] at filter at <console>:58
q6_first_qs: org.apache.spark.rdd.RDD[(String, Float, Int, Int, Int, Int)] = MapPartitionsRDD[12] at map at <console>:65
cols1: Array[String] = Array(Id, cdate, Score, FavoriteCount, ViewCount, AnswerCount)
df_q6_user_info: org.apache.spark.sql.DataFrame = [Id: string, cdate: float ... 4 more fields]


In [4]:
//NEW part3
//q6 find vets
import java.text.SimpleDateFormat
def convert_Id(x: Int) = if (x > 2) 2 else x
val dateFormat = new SimpleDateFormat("yyyy-MM-dd'T'HH:mm:ss.SSS") 
val day = 24*60*60*1000.0
def successMatches3(s: String): (String,Float) = {
  val OwnerUserIdMatcher = """ OwnerUserId=\"(\d+)\" """.r
  val CreationDateMatcher = """ CreationDate=\"(.*?)\" """.r
  val OwnerUserId = OwnerUserIdMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val CreationDate = CreationDateMatcher.findFirstMatchIn(s).map(_.group(1)).getOrElse("0")
  val time = dateFormat.parse(CreationDate).getTime()/day 
  (OwnerUserId,time.toFloat)
  }
val q6_postdate=valid_stack_posts_lines.filter(x => x.contains("OwnerUserId=\"")).map(x => successMatches3(x))

val cols3 : Array[String] = Array("Id", "pcdate")
val df_q6_pc=q6_postdate.toDF(cols3: _*)
//var df_q6_pc=valid_stack_posts_lines.map(x => successMatches3(x)).collect.toSeq.toDF(cols3: _*);

//q6 join df 
val q6_vet_joined=df_q6_pc.join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//deleted .cast("int")
val df_q6_users=q6_vet_joined.withColumn("diff", ($"pcdate"-$"cdate")).na.drop()//.fill(0)?
//deleted >=
.filter(($"diff" > 100) && ($"diff" < 150))
//var df_q6_users=df_q6_pc.join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//.withColumn("diff", ($"pcdate"-$"cdate").cast("int")).na.drop()//.fill(0)?
//.filter(($"diff" >= 100) && ($"diff" < 150));
var df_q6_vets=df_q6_users.groupBy(col("Id")).agg(
      (min($"diff")/100).cast("int").as(("vet")));
var q6_info=df_q6_user_info.join(df_q6_vets, usingColumns=Seq("Id"), joinType="left").na.fill(0);
//--
//var q6_info=df_q6_user_info.join(valid_stack_posts_lines
//                                 .map(x => successMatches3(x)).collect.toSeq.toDF(cols3: _*)
//                                 .join(df_q6_cdate, usingColumns=Seq("Id"), joinType="left")
//.withColumn("diff", ($"pcdate"-$"cdate").cast("int")).na.drop()//.fill(0)?
//.filter(($"diff" >= 100) && ($"diff" < 150)).groupBy(col("Id")).agg(
//      (min($"diff")/100).cast("int").as(("vet"))), usingColumns=Seq("Id"), joinType="left").na.fill(0);

import java.text.SimpleDateFormat
convert_Id: (x: Int)Int
dateFormat: java.text.SimpleDateFormat = java.text.SimpleDateFormat@d2ec523f
day: Double = 8.64E7
successMatches3: (s: String)(String, Float)
q6_postdate: org.apache.spark.rdd.RDD[(String, Float)] = MapPartitionsRDD[14] at map at <console>:62
cols3: Array[String] = Array(Id, pcdate)
df_q6_pc: org.apache.spark.sql.DataFrame = [Id: string, pcdate: float]
q6_vet_joined: org.apache.spark.sql.DataFrame = [Id: string, pcdate: float ... 1 more field]
df_q6_users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Id: string, pcdate: float ... 2 more fields]
df_q6_vets: org.apache.spark.sql.DataFrame = [Id: string, vet: int]
q6_info: org.apache.spark.sql.DataFrame = [Id: string, cdate: float ... 5 more fields]


In [8]:
df_q6_vets.count()

res1: Long = 2027


In [16]:
//q6 reduce
val q6_get_average=q6_info.as[(String, Float,Integer,Integer,Integer,Integer,Integer)]
.rdd.map(x => (x._7,(x._3,x._4,x._5,x._6,1)))
.reduceByKey {
  case ((scL,fcL,vcL,acL,countL), (scR,fcR,vcR,acR,countR)) => 
    (scL+scR,fcL+fcR,vcL+vcR,acL+acR,countL+countR)
}.mapValues { 
  case (sc,fc,vc,ac,count) => 
    (sc.toFloat/count.toFloat,fc.toFloat/count.toFloat,vc.toFloat/count.toFloat,ac.toFloat/count.toFloat)
}
.map(line => (line._1,line._2._1,line._2._2,line._2._3,line._2._4))

q6_get_average: org.apache.spark.rdd.RDD[(Integer, Float, Float, Float, Float)] = MapPartitionsRDD[130] at map at <console>:70


In [6]:
q6_get_average.take(2)

res0: Array[(Integer, Float, Float, Float, Float)] = Array((0,2.1326234,0.5793301,557.671,0.9730719), (1,3.5539055,1.3008801,926.39825,1.2981298))


In [17]:
Array((0,2.131222144030026,0.5792165141918837,556.6446633825944,0.9726014543748533), (1,3.553905390539054,1.300880088008801,926.3982398239824,1.2981298129812981))

res3: Array[(Integer, Float, Float, Float, Float)] = Array((0,2.1326234,0.5793301,557.671,0.9730719), (1,3.5539055,1.3008801,926.39825,1.2981298))


In [ ]:
//vet count
df_q6_vets.count()

In [10]:
//all count
q6_info.count()

res3: Long = 23133
